In [2]:
import os
import numpy as np
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import cv2

# Load the pre-trained ResNet50 model
resnet = ResNet50(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

# Freeze the layers of the pre-trained model
for layer in resnet.layers:
    layer.trainable = False

# Define a new model architecture
model = Sequential([
    resnet,
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer="Adam", loss='binary_crossentropy', metrics=['accuracy'])
how to data augmentation ofr above code
# Load and preprocess your data
def load_data(folder):
    images = []
    labels = []
    for category in os.listdir(folder):
        label = 0 if category == 'Benign' else 1  # Assign label 0 for benign, 1 for malignant
        for filename in os.listdir(os.path.join(folder, category)):
            img = cv2.imread(os.path.join(folder, category, filename))
            img = cv2.resize(img, (150, 150))  # Resize image to desired dimensions
            img = img / 255.0  # Normalize pixel values
            images.append(img)
            labels.append(label)  # Append label for the image
    return np.array(images), np.array(labels)

folder = r"/home/snucse/Desktop/Inbreast Dataset"
images, labels = load_data(folder)

# Split the data into training and testing sets
train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.2, random_state=42)

# Define data augmentation configuration
datagen = ImageDataGenerator(
    rotation_range=20,      # Random rotation in the range [-20, 20] degrees
    width_shift_range=0.2,  # Random horizontal shift by up to 20% of the width
    height_shift_range=0.2, # Random vertical shift by up to 20% of the height
    shear_range=0.2,        # Random shear transformation in the range [-20, 20] degrees
    zoom_range=0.2,         # Random zoom by up to 20%
    horizontal_flip=True,   # Random horizontal flipping
    vertical_flip=True,     # Random vertical flipping
    fill_mode='nearest'     # Fill mode for points outside the input boundaries
)

# Fit the data generator on the training data
datagen.fit(train_images)

# Train the model with data augmentation
history = model.fit(datagen.flow(train_images, train_labels, batch_size=32),
                    steps_per_epoch=len(train_images) // 32,
                    epochs=10,
                    validation_data=(test_images, test_labels))

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_images, test_labels)
print('Test accuracy:', test_acc)



2024-04-10 11:43:35.710688: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-10 11:43:35.755902: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-10 11:43:36.260360: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Epoch 1/10


/home/snucse/.local/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 2s/step - accuracy: 0.5312 - loss: 4.0518 - val_accuracy: 0.6364 - val_loss: 4.9132
Epoch 2/10
1/2 ━━━━━━━━━━━━━━━━━━━━ 0s 532ms/step - accuracy: 0.7273 - loss: 3.5422

2024-04-10 11:43:50.363901: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


...

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 389ms/step - accuracy: 0.6364 - loss: 2.0171 - val_accuracy: 0.6364 - val_loss: 1.3643
Epoch 7/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 988ms/step - accuracy: 0.6470 - loss: 1.0950 - val_accuracy: 0.3636 - val_loss: 0.7092
Epoch 8/10
1/2 ━━━━━━━━━━━━━━━━━━━━ 0s 665ms/step - accuracy: 0.4375 - loss: 1.0406

2024-04-10 11:43:58.736896: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 360ms/step - accuracy: 0.4375 - loss: 1.0406 - val_accuracy: 0.3636 - val_loss: 0.8228
Epoch 9/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step - accuracy: 0.5417 - loss: 1.2652 - val_accuracy: 0.6364 - val_loss: 0.8263
Epoch 10/10
1/2 ━━━━━━━━━━━━━━━━━━━━ 0s 550ms/step - accuracy: 0.4091 - loss: 1.3568

2024-04-10 11:44:01.561878: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 391ms/step - accuracy: 0.4091 - loss: 1.3568 - val_accuracy: 0.6364 - val_loss: 0.8075
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 380ms/step - accuracy: 0.6364 - loss: 0.8075
Test accuracy: 0.6363636255264282
